# **load & understand the data**

In [73]:
import requests

In [74]:
import json

#it imports the request and json libraries

In [75]:
url = "https://raw.githubusercontent.com/mitre/cti/master/enterprise-attack/enterprise-attack.json"

In [76]:
response = requests.get(url)

In [77]:
raw_data = response.json()

#it fetches a json file from a raw GitHub URL using requests.get()

In [78]:
print(type(raw_data))

<class 'dict'>


In [79]:
data = json.loads(response.content)

In [80]:
print(techniques)

{'Tech1': 'Gains administrator access.', 'Tech2': 'Remote bypass with stealth.', 'Tech3': 'Normal execution.'}


#it prints the type and keys of raw data to understand its structure

In [81]:
print(type(raw_data))

<class 'dict'>


In [82]:
print(len(raw_data))

4


In [83]:
print(raw_data.keys())

dict_keys(['type', 'id', 'objects', 'spec_version'])


# **Build Threat scoring logic**

In [43]:
def threat_score(tech):
    score = 5
    name = tech['name'].lower()
    if 'execution' in name: score += 3
    if 'credential' in name: score += 3
    if 'privilege' in name: score += 3
    if 'persistence' in name: score += 3
    if 'lateral' in name: score += 2
    return score

#it assigns a base score of 5 and increments it by specific values when certain keywords (execution, credential, privilege, persistence, lateral) appear in technique name.

In [44]:
technique = {'name': 'credential theft with lateral execution'}
print(threat_score(technique)) 

13


#A dictionary maps descriptive terms to individual scores used to compute a total score for a technique description. 

In [72]:
desc_scores = {
    'administrator': 2,
    'remote': 2,
    'bypass': 2,
    'stealth': 1
}

def score_description(desc):
    return sum(desc_scores[key] for key in desc_scores if key in desc.lower())

def score_techniques(techniques):
    return [(name, score_description(desc)) for name, desc in techniques.items()]
techniques = {
    'Tech1': "Gains administrator access.",
    'Tech2': "Remote bypass with stealth.",
    'Tech3': "Normal execution."
}

results = score_techniques(techniques)
print(results)

[('Tech1', 2), ('Tech2', 5), ('Tech3', 0)]


#score_techniques fun iterates over techniques, calculates each score by matching keywords in the description, and returns a list of pairs.

# **score all techniques**

In [71]:
desc_scores = {
    'administrator': 2,
    'remote': 2,
    'bypass': 2,
    'stealth': 1
}

def score_description(description):
    return sum(desc_scores[key] for key in desc_scores if key in description.lower())

def score_techniques(techniques):
    scored_techniques = [(name, score_description(desc)) for name, desc in techniques.items()]
    return scored_techniques
techniques = {
    'Tech1': "Gains administrator access.",
    'Tech2': "Remote bypass with stealth.",
    'Tech3': "Normal execution."
}
scored_techniques = score_techniques(techniques)
print("Scored techniques:", scored_techniques)

Scored techniques: [('Tech1', 2), ('Tech2', 5), ('Tech3', 0)]


#The code assigns numeric threat scores to security techniques by summing predefined weights for specific keywords found in each technique’s description, producing a ranked list of scored techniques.

# **prioritize threats**

In [70]:
def prioritize_threats(scored_techniques):
    for name, score in sorted(scored_techniques, key=lambda x: x[1], reverse=True):
        print(f"{name} — Score: {score}")
prioritize_threats(scored_techniques)

Credential Dumping — Score: 10
Privilege Escalation — Score: 9
Data Exfiltration — Score: 8.5


#This function sorts the scored attack techniques in descending order of their scores and prints each technique’s name with its score, effectively ranking threats by severity.

# **critical threat filter**

In [69]:
def filter_critical_threats(scored_techniques, threshold=8.9):
    critical_techniques = [(name, score) for name, score in scored_techniques if score >= threshold]
    for name, score in critical_techniques:
        print(f"{name} — Score: {score}")
scored_techniques = [('Credential Dumping', 10), ('Privilege Escalation', 9), ('Data Exfiltration', 8.5)]
filter_critical_threats(scored_techniques)

Credential Dumping — Score: 10
Privilege Escalation — Score: 9


In [89]:
#it filters a list of scored security techniques, returning only those with a score ≥ the given threshold and printing their names and scores.

#The scoring system lets the analyst rank threats by assigning a numeric servity score to each technique.
#It helps prioritize threats by assigning scores, so analysts focus on the most critical ones first.